In [1]:
import datetime
import riskfolio as rp
from lwhf.data.biqquery import BigQueryData, get_Xy
import numpy as np

In [2]:
from lwhf.portfolio.backtest import BackTester

2024-06-13 22:50:45.768159: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-13 22:50:45.771734: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-13 22:50:45.823215: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-13 22:50:47.126195: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
bt = BackTester('2024-05-27',0)

In [4]:
bt.get_all_data()
bt.train_model()
cw = bt.build_portfolio()

✅ Found le-wagon-hedge-fund-data_alpaca_20240604-SP500_Historical_Weekly-2016-01-04-2024-05-27.csv in the local cache.
Epoch 1/10
Epoch 2/10
Epoch 3/10
Epoch 4/10
Epoch 5/10
Epoch 6/10
Epoch 7/10
Epoch 8/10
Epoch 9/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 55ms/step


/home/timboudot/.pyenv/versions/3.10.6/envs/lwhf/lib/python3.10/site-packages/cvxpy/reductions/solvers/solving_chain.py:354: FutureWarning: 
    You specified your problem should be solved by ECOS. Starting in
    CXVPY 1.6.0, ECOS will no longer be installed by default with CVXPY.
    Please either add an explicit dependency on ECOS or switch to our new
    default solver, Clarabel, by either not specifying a solver argument
    or specifying ``solver=cp.CLARABEL``.
    
  warnings.warn(ECOS_DEP_DEPRECATION_MSG, FutureWarning)


The problem doesn't have a solution with actual input parameters


In [5]:
cw

In [6]:
type(cw)

NoneType

In [8]:
as_of = datetime.datetime.strptime(bt.as_of_date, '%Y-%m-%d').date()
returns_df = bt.bq.returns
pred_df = returns_df[returns_df.index.date <= as_of]
pred_X, _ = get_Xy(pred_df)
y_pred = bt.model.predict(pred_X)
cov_df = pred_df.cov()
port = rp.Portfolio(returns=pred_df)
port.mu = y_pred.reshape(-1)
port.cov = cov_df
clean_weights = port.optimization(model='Classic', rm='MV', obj='Sharpe', rf=0, l=0)

14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 41ms/step


/home/timboudot/.pyenv/versions/3.10.6/envs/lwhf/lib/python3.10/site-packages/cvxpy/reductions/solvers/solving_chain.py:354: FutureWarning: 
    You specified your problem should be solved by ECOS. Starting in
    CXVPY 1.6.0, ECOS will no longer be installed by default with CVXPY.
    Please either add an explicit dependency on ECOS or switch to our new
    default solver, Clarabel, by either not specifying a solver argument
    or specifying ``solver=cp.CLARABEL``.
    
  warnings.warn(ECOS_DEP_DEPRECATION_MSG, FutureWarning)


The problem doesn't have a solution with actual input parameters


In [19]:
if np.all(y_pred < 0):
    port.mu = pred_df.mean().values.reshape(-1)
else:
    port.mu = y_pred.reshape(-1)

In [20]:
clean_weights = port.optimization(model='Classic', rm='MV', obj='Sharpe', rf=0, l=0)

In [24]:
clean_weights.round(5).value_counts()

weights
0.00000    407
0.00046      1
0.00053      1
0.00709      1
0.01313      1
0.01322      1
0.04041      1
0.04078      1
0.05327      1
0.07859      1
0.08831      1
0.11018      1
0.11874      1
0.21638      1
0.21891      1
Name: count, dtype: int64

In [16]:
port.mu

array([-0.01616495, -0.02127801, -0.02213686, -0.01623977, -0.02342338,
       -0.02310999, -0.02529256, -0.02373177, -0.01021907, -0.02012238,
       -0.01930471, -0.02495392, -0.02130469, -0.01747159, -0.00978871,
       -0.0172876 , -0.01791363, -0.0180071 , -0.02219937, -0.01797451,
       -0.0256823 , -0.01792432, -0.02722193, -0.01762173, -0.02011934,
       -0.02198169, -0.01347779, -0.01863882, -0.0201191 , -0.01953986,
       -0.01604676, -0.01666996, -0.02070881, -0.01899526, -0.02017309,
       -0.02435106, -0.02014208, -0.02445693, -0.01534222, -0.00995695,
       -0.02155546, -0.0174855 , -0.0143015 , -0.01713491, -0.01803886,
       -0.01616532, -0.01745285, -0.02288983, -0.02359639, -0.01494319,
       -0.02774274, -0.02359943, -0.02235573, -0.02546497, -0.02624594,
       -0.02012891, -0.01729604, -0.02063434, -0.02838753, -0.01964217,
       -0.01432992, -0.01742778, -0.0230848 , -0.01604916, -0.01903107,
       -0.0247683 , -0.01888376, -0.01687906, -0.01993924, -0.02

In [4]:
pred_df.mean().shape

NameError: name 'pred_df' is not defined

symbol,A,AAL,AAP,AAPL,ABBV,ABT,ACN,ADBE,ADI,ADM,...,WYNN,XEL,XOM,XRAY,XRX,XYL,YUM,ZBH,ZION,ZTS
timestamp,,,,,,,,,,,,,,,,,,,,,
2016-01-11 05:00:00+00:00,-0.038352,-0.039881,-0.008725,0.001753,0.030368,-0.003196,0.010896,0.015026,-0.003201,-0.083745,...,0.010911,0.008845,0.038693,-0.008773,-0.051095,-0.040442,-0.015409,-0.004628,-0.091870,-0.054490
2016-01-18 05:00:00+00:00,0.023444,0.036120,0.000970,0.044168,0.025985,-0.012580,0.029918,0.005159,0.031313,0.054586,...,0.013706,0.003562,-0.013019,0.018233,-0.003297,0.031837,0.028171,-0.019094,-0.037153,0.000461
2016-01-25 05:00:00+00:00,-0.008689,-0.029133,0.052759,-0.040229,-0.066803,-0.054459,0.032277,-0.005578,0.048268,0.063798,...,0.138077,0.043407,0.016717,0.023818,0.074972,0.056421,0.049145,0.001109,0.054393,-0.008065
2016-02-01 05:00:00+00:00,-0.042762,-0.057451,-0.053338,-0.034107,-0.032423,-0.011625,-0.062251,-0.111410,-0.075009,-0.036209,...,-0.080636,0.034275,0.028645,0.001698,-0.029744,0.019750,-0.036065,-0.042817,-0.046296,-0.049710
2016-02-08 05:00:00+00:00,0.004994,0.029116,-0.022023,-0.000319,-0.010166,-0.007485,-0.043144,-0.031944,-0.003613,-0.047549,...,0.116782,-0.017961,0.011863,-0.072216,-0.072939,0.010366,-0.034690,-0.033996,-0.029589,-0.011733
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-04-29 04:00:00+00:00,0.012415,-0.002161,0.015571,0.083166,0.026125,-0.015159,-0.013961,0.018050,-0.011586,-0.015474,...,0.008657,0.005374,-0.016616,-0.074583,-0.045714,0.036329,-0.052409,0.019355,0.044540,0.054602
2024-05-06 04:00:00+00:00,0.073933,0.039711,-0.016774,-0.001800,-0.018560,-0.010954,0.008627,-0.008001,0.037870,0.064391,...,-0.003309,0.022304,0.016897,-0.014139,0.013473,0.042198,0.024416,-0.002877,0.016048,0.011791
2024-05-13 04:00:00+00:00,0.029848,0.022917,-0.006397,0.037258,0.035272,-0.006206,-0.008945,0.002364,0.033255,-0.021594,...,0.004150,0.001082,0.014242,0.013984,0.030281,0.002238,0.029356,-0.009315,0.014440,0.029697
